In [1]:
import tensorflow as tf

2025-11-29 17:11:38.135746: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-29 17:11:38.196830: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-29 17:11:38.215293: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-29 17:11:38.338008: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-29 17:11:40.132840: W tensorflow/compiler/tf2

In [2]:
from mobilenetv2ssd.core.config import load_config

In [3]:
from mobilenetv2ssd.models.ssd.ops.match_ops_tf import hard_negative_mining

In [4]:
from typing import Any

## Config Files To Test Orchestration

In [5]:
main_cfg_path = "configs/train/default.yaml"
model_cfg_path = "configs/model/mobilenetv2_ssd_voc.yaml"
data_cfg_path = "configs/data/voc_224.yaml"
eval_cfg_path = "configs/eval/default.yaml"

In [6]:
config = load_config(main_cfg_path,model_cfg_path,data_cfg_path,eval_cfg_path)

In [7]:
config['train']

{'seed': 1337,
 'device': 'auto',
 'epochs': 50,
 'batch_size': 3,
 'grad_accum_steps': 1,
 'optimizer': {'name': 'sgd',
  'lr': 0.001,
  'momentum': 0.9,
  'weight_decay': 0.0005,
  'nesterov': True},
 'scheduler': {'name': 'cosine', 'warmup': {'epochs': 5, 'factor': 0.1}},
 'loss': {'cls_loss_type': 'ce_softmax',
  'from_logits': True,
  'ignore_index': -1,
  'label_smoothing': 0.0,
  'use_sigmoid': False,
  'class_weights': None,
  'reg_loss_type': 'smooth_l1',
  'smooth_l1_beta': 1.0,
  'bbox_norm': 'none',
  'alpha': 1.0,
  'gamma': 2.0,
  'cls_weight': 1.0,
  'reg_weight': 1.0,
  'normalization': {'type': 'num_pos', 'epsilon': '1e-6'}},
 'sampler': {'neg_pos_ratio': 3.0, 'min_neg': 0, 'max_neg': None},
 'targets': {'filters': {'min_box_area': 0.0,
   'clip_to_image': True,
   'drop_invalid': True}},
 'diagnostics': False}

## Helper Functions For Orchestration

In [8]:
def _extract_information_from_train_config(config : dict[str, Any]):
    train_config = config['train']
    sampler_config = train_config.get("sampler",{})
    target_config = {
        "neg_pos_ratio": sampler_config.get("neg_pos_ratio",3.0),
        "min_neg":  sampler_config.get("min_neg",0),
        "max_neg": sampler_config.get("max_neg",None),
    }

    return target_config

In [9]:
_extract_information_from_train_config(config)

{'neg_pos_ratio': 3.0, 'min_neg': 0, 'max_neg': None}

## Orchestration Function

In [10]:
def select_hard_negatives(config: dict[str, Any], conf_loss: tf.Tensor, positive_mask: tf.Tensor, negative_mask: tf.Tensor):
    # This function will orchestrate the hard negative mining that needs to be done
    # Steps:
    # 1. Get values from the config
    # 2. Calculate Hard negatives
    sampler_config = _extract_information_from_train_config(config)

    selected_negative_mask, selected_negative_indices = tf.map_fn(lambda inputs: hard_negative_mining(conf_loss = inputs[0],pos_mask = inputs[1],neg_mask = inputs[2],neg_ratio = sampler_config['neg_pos_ratio'], min_neg = sampler_config['min_neg'], max_neg = sampler_config['max_neg']),
              elems = (conf_loss,positive_mask,negative_mask), 
              fn_output_signature = ( tf.TensorSpec(shape=(None,), dtype=tf.bool),tf.TensorSpec(shape=(None,1),dtype=tf.int32))
             )
         
    return selected_negative_mask, selected_negative_indices
    

In [11]:
conf_loss_batched = tf.constant([
    # image 0
    [0.2, 1.5, 0.1, 2.0, 0.3, 0.8],
    # image 1
    [0.9, 0.4, 1.8, 0.2, 0.05, 2.5],
], dtype=tf.float32)

pos_mask_batched = tf.constant([
    [False, True,  False, False, True,  False],  # image 0
    [False, False, True,  False, False, True],   # image 1
], dtype=tf.bool)

neg_mask_batched = tf.logical_not(pos_mask_batched)

I0000 00:00:1764392884.217559   65573 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1764392884.310882   65573 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1764392884.310942   65573 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1764392884.312922   65573 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1764392884.312986   65573 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [12]:
conf_loss_batched

<tf.Tensor: shape=(2, 6), dtype=float32, numpy=
array([[0.2 , 1.5 , 0.1 , 2.  , 0.3 , 0.8 ],
       [0.9 , 0.4 , 1.8 , 0.2 , 0.05, 2.5 ]], dtype=float32)>

In [13]:
select_hard_negatives(config,conf_loss_batched,pos_mask_batched,neg_mask_batched)

(<tf.Tensor: shape=(2, 6), dtype=bool, numpy=
 array([[ True, False,  True,  True, False,  True],
        [ True,  True, False,  True,  True, False]])>,
 <tf.Tensor: shape=(2, 4, 1), dtype=int32, numpy=
 array([[[3],
         [5],
         [0],
         [2]],
 
        [[0],
         [1],
         [3],
         [4]]], dtype=int32)>)